In [1]:
import pandas as pd

In [2]:
# Load in mta 2018 pickle file
mta_data = pd.read_pickle('../dump/mta_data')

In [3]:
#  Fix EXITS column name
mta_data.rename(columns={'EXITS                                                               ':'EXITS'}, inplace=True)

In [4]:
# Create DateTime column
mta_data['DATETIME'] = pd.to_datetime(mta_data.DATE + ' ' + mta_data.TIME)
mta_data['DATE']= pd.to_datetime(mta_data['DATE'])

In [5]:
# Convert Date object to Datetime
mta_data['DATE']= pd.to_datetime(mta_data['DATE'])

In [6]:
# Create Weekday from newly create DateTime column
mta_data['WEEKDAY'] = mta_data['DATETIME'].dt.day_name()

In [7]:
# Identify Daytime
mta_data['DAYTYPE'] = mta_data['WEEKDAY'].apply(lambda x: 'WEEKEND' if x == 'Sunday' else 'WEEKDAY')

In [8]:
# Create ID for each turnstile
mta_data['UNIQUEID'] = mta_data['STATION'] + '_' + mta_data['UNIT'] + '_' + mta_data['C/A'] + '_' + mta_data['SCP']

In [9]:
# Create Time of Day
mta_data['TIMEOFDAY'] = mta_data['DATETIME'].dt.hour

In [10]:
# Daily Counts by Turnstile
mta_data['ENTRYDIFF'] = mta_data['ENTRIES'].diff()
mta_data['EXITDIFF'] = mta_data['EXITS'].diff()

In [11]:
mta_data['DURATION'] = mta_data.DATETIME.diff().apply(lambda x: x.total_seconds()/3600)

In [12]:
# Drop Rows without changes
mta_data.dropna(subset=['ENTRYDIFF', 'EXITDIFF'], inplace=True)

In [13]:
# Drop First Entry Per Turnstile
mta_data = mta_data[mta_data.DATETIME >= mta_data.DATE.min() + pd.DateOffset(days=1)]

In [14]:
# Normalize floats to int values
mta_data['ENTRYDIFF'] = mta_data['ENTRYDIFF'].astype(int)
mta_data['EXITDIFF'] = mta_data['EXITDIFF'].astype(int)
mta_data['TRAFFIC'] = mta_data['ENTRYDIFF'] + mta_data['EXITDIFF']

In [15]:
mta_data['AVG_TRAFFIC'] = mta_data['TRAFFIC'] / mta_data['DURATION']

In [16]:
# Drop ENTRYDIFF AND EXITDIFF Rows with extreme values
mta_data = mta_data[mta_data.ENTRYDIFF <= 10000]
mta_data = mta_data[mta_data.ENTRYDIFF > 0]
mta_data = mta_data[mta_data.EXITDIFF <= 10000]
mta_data = mta_data[mta_data.EXITDIFF > 0]

In [17]:
# Export pickle of processed mta data
mta_data.to_pickle('../dump/mta_data_processed')